# Basic Fluid Properties with NeqSim

This notebook provides a foundational introduction to using NeqSim for calculating basic thermodynamic and transport properties of various fluids. It covers:

1.  Defining a fluid system (composition, equation of state).
2.  Setting thermodynamic conditions (pressure, temperature).
3.  Performing flash calculations to determine phase behavior.
4.  Retrieving key properties for different phases (gas, liquid, aqueous).
5.  Demonstrating unit handling.
6.  Generating simple plots of properties over a range of conditions.

## Table of Contents
1.  [Installation](#1.-Installation)
2.  [Core Concepts](#2.-Core-Concepts)
3.  [Examples of Fluid Property Calculations](#3.-Examples-of-Fluid-Property-Calculations)
    *   3.1. Simple Dry Gas
    *   3.2. Black Oil System (Vapor-Liquid Equilibrium)
    *   3.3. Brine (Aqueous) System
    *   3.4. Multi-Component Hydrocarbon Mixture
4.  [Generating Property Curves (Pressure Scan)](#4.-Generating-Property-Curves-(Pressure-Scan))
5.  [Summary](#5.-Summary)

## 1. Installation

First, ensure NeqSim is installed in your Colab environment. The `--quiet` flag prevents verbose output during installation.

In [ ]:
!pip install neqsim --quiet

## 2. Core Concepts

Before diving into examples, let's briefly review the core NeqSim concepts used for property calculations:

*   `fluid(model_name)`: Creates a fluid object. `model_name` typically refers to an Equation of State (EOS) like `'pr'` (Peng-Robinson) or `'srk'` (Soave-Redlich-Kwong), or specialized models like `'phreeqc'` for aqueous systems.
*   `addComponent(component_name, molar_fraction)`: Adds a chemical component to the fluid. Molar fractions can be specified as a sum of 1.0 or as relative amounts.
*   `setPressure(value, unit)`: Sets the system pressure. Common units: `'bara'`, `'bar'`, `'kPa'`, `'psi'`.
*   `setTemperature(value, unit)`: Sets the system temperature. Common units: `'K'`, `'C'`, `'F'`.
*   `runFlash()`: Performs a flash calculation. This is the crucial step that determines the number of phases present at the given conditions and their respective compositions and properties.
*   `getPhase(phase_name)`: Retrieves a specific phase object (e.g., `'gas'`, `'oil'`, `'aqueous'`, `'solid'`). You can then query properties of that phase.
*   `get...('unit')`: Most property getter methods allow you to specify the desired unit for the output (e.g., `getDensity('kg/m3')`).

Let's import the necessary libraries:

In [ ]:
from neqsim.thermo import fluid, phase, system
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 3. Examples of Fluid Property Calculations

We will now demonstrate how to calculate properties for different types of fluids.

### 3.1. Simple Dry Gas

Let's start with a simple natural gas mixture (methane and ethane) and calculate its properties at a given pressure and temperature.

In [ ]:
print("--- Example 1: Simple Dry Gas ---")

# 1. Define the fluid using Peng-Robinson Equation of State (PR EOS)
gas_fluid = fluid('pr')

# 2. Add components and their molar compositions
gas_fluid.addComponent("methane", 0.9) # 90 mol% methane
gas_fluid.addComponent("ethane", 0.1)  # 10 mol% ethane

# 3. Set the thermodynamic conditions
gas_fluid.setPressure(150.0, 'bara') # 150 bara (absolute pressure)
gas_fluid.setTemperature(320.0, 'K') # 320 K (approx 47 deg C)

# 4. Perform a flash calculation
gas_fluid.runFlash()

# 5. Display results
print(f"\nSystem Pressure: {gas_fluid.getPressure('bara'):.2f} bara")
print(f"System Temperature: {gas_fluid.getTemperature('K'):.2f} K")
print(f"Number of phases: {gas_fluid.getNumberOfPhases()}")

# Check if the gas phase exists before querying its properties
if gas_fluid.hasPhase('gas') and gas_fluid.getPhase('gas') is not None:
    gas_phase = gas_fluid.getPhase('gas')
    print(f"  Gas density: {gas_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Gas viscosity: {gas_phase.getViscosity('cP'):.4f} cP")
    print(f"  Gas Z-factor: {gas_phase.getZ():.4f}")
    print(f"  Gas molar volume: {gas_phase.getMolarVolume('m3/mole'):.6f} m3/mole")
    print(f"  Gas molecular weight: {gas_phase.getMolarMass('kg/mole')*1000:.2f} g/mole")
    print(f"  Gas heat capacity (Cp): {gas_phase.getHeatCapacityCp('J/molK'):.2f} J/molK")
    print(f"  Gas enthalpy: {gas_phase.getEnthalpy('J/mol'):.2f} J/mol")
else:
    print("  No gas phase found at these conditions.")

### 3.2. Black Oil System (Vapor-Liquid Equilibrium)

This example demonstrates a fluid that can exist in both vapor (gas) and liquid (oil) phases. We'll show its behavior above and below its bubble point pressure.

In [ ]:
print("\n--- Example 2: Black Oil System (Gas + Oil) ---")

oil_fluid = fluid('pr')
oil_fluid.addComponent("methane", 60.0) # Relative molar amounts
oil_fluid.addComponent("nC10", 40.0) # Normal decane as a heavy component
oil_fluid.setTemperature(370.0, 'K') # Constant reservoir temperature (approx 97 deg C)

# --- Scenario 1: Above Bubble Point (Single-Phase Oil) ---
oil_fluid.setPressure(250.0, 'bara')
oil_fluid.runFlash()

print(f"\nScenario 1: Pressure = {oil_fluid.getPressure('bara'):.2f} bara")
print(f"  Number of phases: {oil_fluid.getNumberOfPhases()}")

bubble_point_pressure = oil_fluid.getBubblePointPressure('bara')
print(f"  Calculated Bubble Point Pressure: {bubble_point_pressure:.2f} bara")

if oil_fluid.hasPhase('oil') and oil_fluid.getPhase('oil') is not None:
    oil_phase = oil_fluid.getPhase('oil')
    print(f"  Oil density: {oil_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Oil viscosity: {oil_phase.getViscosity('cP'):.4f} cP")
    # Note: Bo and Rs are typically calculated relative to standard conditions, not just a single phase's properties
    # These are more relevant in the 'Generating PVT Tables' notebook.
else:
    print("  No oil phase or not single-phase oil at these conditions.")

# --- Scenario 2: Below Bubble Point (Two-Phase Gas + Oil) ---
oil_fluid.setPressure(100.0, 'bara') # Well below the bubble point
oil_fluid.runFlash()

print(f"\nScenario 2: Pressure = {oil_fluid.getPressure('bara'):.2f} bara")
print(f"  Number of phases: {oil_fluid.getNumberOfPhases()}")

if oil_fluid.hasPhase('oil') and oil_fluid.getPhase('oil') is not None:
    oil_phase = oil_fluid.getPhase('oil')
    print(f"  Oil molar fraction: {oil_phase.getMolarFraction():.4f}")
    print(f"  Oil density: {oil_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Oil viscosity: {oil_phase.getViscosity('cP'):.4f} cP")
else:
    print("  Oil phase not present or not identified.")

if oil_fluid.hasPhase('gas') and oil_fluid.getPhase('gas') is not None:
    gas_phase = oil_fluid.getPhase('gas')
    print(f"  Gas molar fraction: {gas_phase.getMolarFraction():.4f}")
    print(f"  Gas density: {gas_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Gas viscosity: {gas_phase.getViscosity('cP'):.4f} cP")
    print(f"  Gas Z-factor: {gas_phase.getZ():.4f}")
else:
    print("  Gas phase not present or not identified.")

### 3.3. Brine (Aqueous) System

NeqSim can also model water and dissolved salts. We use the `'phreeqc'` model for aqueous systems which provides more accurate water properties, especially with dissolved solids.

In [ ]:
print("\n--- Example 3: Brine System ---")

water_fluid = fluid('phreeqc') # Using PHREEQC model for water properties
water_fluid.addComponent("H2O", 1.0)
water_fluid.addComponent("NaCl", 0.05) # 5% molar NaCl (approx 3.5 wt% salinity)

water_fluid.setPressure(100.0, 'bara')
water_fluid.setTemperature(350.0, 'K')

water_fluid.runFlash()

if water_fluid.hasPhase('aqueous') and water_fluid.getPhase('aqueous') is not None:
    aqueous_phase = water_fluid.getPhase('aqueous')
    print(f"  Water density: {aqueous_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Water viscosity: {aqueous_phase.getViscosity('cP'):.4f} cP")
    print(f"  pH: {aqueous_phase.getpH():.2f}")
    print(f"  Conductivity: {aqueous_phase.getConductivity('S/m'):.2f} S/m")
else:
    print("  No aqueous phase found.")

### 3.4. Multi-Component Hydrocarbon Mixture

Let's look at a more complex mixture with light, intermediate, and heavy components to see how the flash calculation handles it.

In [ ]:
print("\n--- Example 4: Multi-Component Hydrocarbon Mixture ---")

multi_comp_fluid = fluid('pr')
multi_comp_fluid.addComponent("methane", 80.0)
multi_comp_fluid.addComponent("ethane", 5.0)
multi_comp_fluid.addComponent("propane", 3.0)
multi_comp_fluid.addComponent("n-butane", 2.0)
multi_comp_fluid.addComponent("n-hexane", 2.0)
multi_comp_fluid.addComponent("n-decane", 5.0)
multi_comp_fluid.addComponent("nC20", 3.0) # A heavier component

multi_comp_fluid.setPressure(180.0, 'bara')
multi_comp_fluid.setTemperature(360.0, 'K')
multi_comp_fluid.runFlash()

print(f"\nSystem Pressure: {multi_comp_fluid.getPressure('bara'):.2f} bara")
print(f"System Temperature: {multi_comp_fluid.getTemperature('K'):.2f} K")
print(f"Number of phases: {multi_comp_fluid.getNumberOfPhases()}")

if multi_comp_fluid.hasPhase('gas') and multi_comp_fluid.getPhase('gas') is not None:
    gas_phase = multi_comp_fluid.getPhase('gas')
    print(f"  Gas molar fraction: {gas_phase.getMolarFraction():.4f}")
    print(f"  Gas density: {gas_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Gas viscosity: {gas_phase.getViscosity('cP'):.4f} cP")
else:
    print("  No gas phase found.")

if multi_comp_fluid.hasPhase('oil') and multi_comp_fluid.getPhase('oil') is not None:
    oil_phase = multi_comp_fluid.getPhase('oil')
    print(f"  Oil molar fraction: {oil_phase.getMolarFraction():.4f}")
    print(f"  Oil density: {oil_phase.getDensity('kg/m3'):.2f} kg/m3")
    print(f"  Oil viscosity: {oil_phase.getViscosity('cP'):.4f} cP")
else:
    print("  No oil phase found.")

## 4. Generating Property Curves (Pressure Scan)

Often, you need to see how properties change over a range of conditions. We can iterate through different pressures (or temperatures) and plot the results. Let's use our `black_oil` fluid and plot its phase densities and viscosities as pressure changes.

In [ ]:
print("\n--- Example 5: Property Curves (Pressure Scan) ---")

oil_fluid_scan = fluid('pr')
oil_fluid_scan.addComponent("methane", 60.0)
oil_fluid_scan.addComponent("nC10", 40.0)
oil_fluid_scan.setTemperature(370.0, 'K') # Keep temperature constant

pressures = np.linspace(10.0, 300.0, 50) # 50 points from 10 to 300 bara

data = []
for p in pressures:
    oil_fluid_scan.setPressure(p, 'bara')
    oil_fluid_scan.runFlash()

    row = {'Pressure (bara)': p}

    # Get properties for oil phase
    if oil_fluid_scan.hasPhase('oil') and oil_fluid_scan.getPhase('oil') is not None:
        oil_phase = oil_fluid_scan.getPhase('oil')
        row['Oil Density (kg/m3)'] = oil_phase.getDensity('kg/m3')
        row['Oil Viscosity (cP)'] = oil_phase.getViscosity('cP')
    else:
        row['Oil Density (kg/m3)'] = np.nan
        row['Oil Viscosity (cP)'] = np.nan

    # Get properties for gas phase
    if oil_fluid_scan.hasPhase('gas') and oil_fluid_scan.getPhase('gas') is not None:
        gas_phase = oil_fluid_scan.getPhase('gas')
        row['Gas Density (kg/m3)'] = gas_phase.getDensity('kg/m3')
        row['Gas Viscosity (cP)'] = gas_phase.getViscosity('cP')
        row['Gas Z-factor'] = gas_phase.getZ()
    else:
        row['Gas Density (kg/m3)'] = np.nan
        row['Gas Viscosity (cP)'] = np.nan
        row['Gas Z-factor'] = np.nan

    data.append(row)

df = pd.DataFrame(data)

print("\nDataFrame of properties (first 5 rows):\n")
print(df.head())
print("\n...\n")
print("DataFrame of properties (last 5 rows):\n")
print(df.tail())

# --- Plotting the results ---
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(df['Pressure (bara)'], df['Oil Density (kg/m3)'], label='Oil Density', color='blue')
plt.plot(df['Pressure (bara)'], df['Gas Density (kg/m3)'], label='Gas Density', color='red', linestyle='--')
plt.xlabel('Pressure (bara)')
plt.ylabel('Density (kg/m3)')
plt.title('Fluid Densities vs. Pressure')
plt.grid(True)
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(df['Pressure (bara)'], df['Oil Viscosity (cP)'], label='Oil Viscosity', color='blue')
plt.plot(df['Pressure (bara)'], df['Gas Viscosity (cP)'], label='Gas Viscosity', color='red', linestyle='--')
plt.xlabel('Pressure (bara)')
plt.ylabel('Viscosity (cP)')
plt.title('Fluid Viscosities vs. Pressure')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(df['Pressure (bara)'], df['Gas Z-factor'], label='Gas Z-factor', color='green')
plt.xlabel('Pressure (bara)')
plt.ylabel('Z-factor')
plt.title('Gas Z-factor vs. Pressure')
plt.grid(True)
plt.legend()
plt.show()

## 5. Summary

This notebook has provided a basic overview of how to use NeqSim to:

*   Define fluid compositions.
*   Set system conditions (P, T).
*   Run flash calculations to determine phase behavior.
*   Retrieve fundamental properties for different phases (density, viscosity, Z-factor, etc.).
*   Generate and visualize property curves over a range of pressures.

This foundational knowledge is crucial for more advanced applications in NeqSim, such as PVT table generation for reservoir simulation, phase envelope calculations, and process unit modeling. You are now equipped to explore more complex thermodynamic problems with NeqSim!